MIPCL: https://mipcl-cpp.appspot.com/
>MIPCL is currently one of the fastest non-commercial mixed integer programming solvers. It can be used together with a modeling tool named MIPshell.


This kernel uses the code from: [Benchmark MIP solvers(Draft)](https://www.kaggle.com/golubev/benchmark-mip-solvers-draft)

Setup

In [ ]:
%%bash
# Install Python 2.7 because the shared library of MIPCL for Python3 is broken (link error).
git clone git://github.com/yyuu/pyenv.git ~/.pyenv
echo 'export PYENV_ROOT="$HOME/.pyenv"' >> ~/.bash_profile
echo 'export PATH="$PYENV_ROOT/bin:$PATH"' >> ~/.bash_profile
echo 'eval "$(pyenv init -)"' >> ~/.bash_profile
source ~/.bash_profile
apt-get install -y libssl-dev libreadline-dev
pyenv install 2.7.17
pyenv local 2.7.17
pip install pandas numpy

# Install MIPCL(mipcl_py module)
wget https://mipcl-cpp.appspot.com/static/download/mipcl-py-2.6.1.linux-x86_64.tar.gz
tar --exclude='*docs' -xzvf mipcl-py-2.6.1.linux-x86_64.tar.gz # exclude docs directory due to `too many nested subdirectories error` in kaggle kernel
rm -f ./mipcl_py/mipshell/mipcl.so
ln -s mipcl-py2.so ./mipcl_py/mipshell/mipcl.so # Use mipcl-py2

Benchmark code from https://www.kaggle.com/golubev/benchmark-mip-solvers-draft

In [ ]:
%%bash
source ~/.bash_profile
python <<__EOF__
from __future__ import print_function
import time
import numpy as np
import pandas as pd
import mipcl_py.mipshell.mipshell as mipshell

def get_days(assigned_days, n_people):
    days = np.zeros(assigned_days.max(), int)
    for i, r in enumerate(assigned_days):
        days[r-1] += n_people[i]
    return days


def example_mipcl(desired, n_people):
    def accounting_penalty(day, next_day):
        return (day - 125.0) * (day**(0.5 + abs(day - next_day) / 50.0)) / 400.0
    FAMILY_COST = np.asarray([0,50,50,100,200,200,300,300,400,500])
    MEMBER_COST = np.asarray([0, 0, 9,  9,  9, 18, 18, 36, 36,235])
    num_days = desired.max()
    num_families = desired.shape[0]
    solver = mipshell.Problem(name='Santa2019 only preference')
    C, B, I = {}, {}, {}

    for fid, choices in enumerate(desired):
        for cid in range(10):
            B[fid, choices[cid]-1] = mipshell.Var(type=mipshell.BIN, lb=0.0, ub=1.0)
            C[fid, choices[cid]-1] = FAMILY_COST[cid] + n_people[fid] * MEMBER_COST[cid]

    for day in range(num_days):
        I[day] = mipshell.Var(type=mipshell.INT, lb=125, ub=300)
        mipshell.sum_(n_people[fid]*B[fid, day] for fid in range(num_families) if (fid,day) in B) == I[day]

    for fid in range(num_families):
        mipshell.sum_(B[fid, day] for day in range(num_days) if (fid,day) in B) == 1

    objective = mipshell.sum_(C[fid, day]*B[fid, day] for fid, day in B)

    solver.minimize(objective)
    solver.optimize(silent=False, gap=0.0)
    if solver.is_solution:
        print("Result: ", solver.getObjVal())
        assigned_days = np.zeros(num_families, int)
        for fid, day in B:
            if B[fid, day].val > 0.5:
                assigned_days[fid] = day + 1
        return assigned_days
    else:
        print("Failed", solver.is_solution, solver.is_infeasible, solver.isPureLP)
        return None


def save(assigned_days):
    with open("submission.csv", "w") as f:
        f.write("family_id,assigned_day\n")
        for fid, v in enumerate(assigned_days):
            f.write("{},{}\n".format(fid, v))


if __name__ == "__main__":
    ds = pd.read_csv('../input/santa-workshop-tour-2019/family_data.csv')
    t = time.time()
    ret = example_mipcl(ds.values[:,1:11], ds.values[:,11])
    if ret is not None:
        save(ret)
    print("Elapsed time", time.time() - t)
__EOF__

43622 is the optimal preference cost mentioned in https://www.kaggle.com/mihaild/lower-bound-on-preference-cost#685643